<a href="https://colab.research.google.com/github/nicole-yoong/Portfolio/blob/main/Pyspark/Pyspark_Fundamental_Linear_Regression_NHL_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 56.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b0426ca803b4503c2378a153c446c82c703a4c5c1447f86b70fc4f52e7468e01
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


# Import packages

In [ ]:
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import col,isnan, when, count


In [ ]:
spark = SparkSession.builder.master("local[2]").appName("California-Housing").getOrCreate()

# Reading dataset

In [ ]:
#Read the CSV file into a Spark dataframe, similar to df.head() in Pandas
file_location = "/content/drive/MyDrive/Python/Case Studies/Datasets/california_housing.csv"
df = spark.read.format("csv").option("inferSchema",True).option("header", True).load(file_location)
display(df)

DataFrame[longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double, ocean_proximity: string]

In [ ]:
#Save the file to parquet format as it is a file format that includes metadata about the column data types, offers file compression
df.write.save("/content/drive/MyDrive/Python/Case Studies/Datasets/california_housing.parquet", format='parquet')
df = spark.read.load("/content/drive/MyDrive/Python/Case Studies/Datasets/california_housing.parquet")
display(df)

## Spark to Pandas dataframe

In [ ]:
df.limit(10).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


## Spark to SparkSQL dataframe

In [ ]:
df.createOrReplaceTempView("df")
spark.sql("select * from df").show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

# Missing values 

In [ ]:
# Find Count of Null, None, NaN of All DataFrame Columns
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [ ]:
df = df.dropna()
df.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

# Data exploration

In [ ]:
df.take(5)

[Row(longitude=-122.23, latitude=37.88, housing_median_age=41.0, total_rooms=880.0, total_bedrooms=129.0, population=322.0, households=126.0, median_income=8.3252, median_house_value=452600.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.22, latitude=37.86, housing_median_age=21.0, total_rooms=7099.0, total_bedrooms=1106.0, population=2401.0, households=1138.0, median_income=8.3014, median_house_value=358500.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.24, latitude=37.85, housing_median_age=52.0, total_rooms=1467.0, total_bedrooms=190.0, population=496.0, households=177.0, median_income=7.2574, median_house_value=352100.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.25, latitude=37.85, housing_median_age=52.0, total_rooms=1274.0, total_bedrooms=235.0, population=558.0, households=219.0, median_income=5.6431, median_house_value=341300.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.25, latitude=37.85, housing_median_age=52.0, total_rooms=1627.0, total_bedrooms=280.0,

In [ ]:
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [ ]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



## Columns

In [ ]:
df.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']

In [ ]:
df.select('longitude','latitude').show(10)

+---------+--------+
|longitude|latitude|
+---------+--------+
|  -122.23|   37.88|
|  -122.22|   37.86|
|  -122.24|   37.85|
|  -122.25|   37.85|
|  -122.25|   37.85|
|  -122.25|   37.85|
|  -122.25|   37.84|
|  -122.25|   37.84|
|  -122.26|   37.84|
|  -122.25|   37.84|
+---------+--------+
only showing top 10 rows



In [ ]:
df.withColumnRenamed("longitude","long").show(10)

+-------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|   long|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+-------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|-122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|-122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|-122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|-122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|-122.25|   37.85|              52.0|     1627.0

In [ ]:
df.withColumnRenamed("longitude","long").withColumnRenamed("latitude","lat").show(10)

+-------+-----+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|   long|  lat|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+-------+-----+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|-122.23|37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|-122.22|37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|-122.24|37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|-122.25|37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|-122.25|37.85|              52.0|     1627.0|         280.0|     565

## Sort

In [ ]:
df.sort("median_house_value").show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -117.86|   34.24|              52.0|      803.0|         267.0|     628.0|     225.0|       4.1932|           14999.0|         INLAND|
|  -123.17|   40.31|              36.0|       98.0|          28.0|      18.0|       8.0|        0.536|           14999.0|         INLAND|
|  -117.02|    36.4|              19.0|      619.0|         239.0|     490.0|     164.0|          2.1|           14999.0|         INLAND|
|  -122.74|   39.71|              16.0|      255.0|          73.0|      85.0|      38.0|       1.6607|           14999.0|         INLAND|
|  -118.33|   34.15|              

In [ ]:
from pyspark.sql import functions as F
df.sort(F.desc("median_house_value")).show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.82|              52.0|     3494.0|         396.0|    1192.0|     383.0|      12.3804|          500001.0|       NEAR BAY|
|  -121.97|   37.87|               4.0|     1029.0|         126.0|     416.0|     122.0|      13.4883|          500001.0|         INLAND|
|  -122.25|   37.87|              52.0|      609.0|         236.0|    1349.0|     250.0|       1.1696|          500001.0|       NEAR BAY|
|  -122.27|    37.8|              52.0|      249.0|          78.0|     396.0|      85.0|       1.2434|          500001.0|       NEAR BAY|
|   -122.0|   37.86|              

## Filter

In [ ]:
df.filter((df.median_house_value > 500000) & (df.total_rooms >3000) & (df.ocean_proximity == 'INLAND')).show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -121.96|   37.85|              10.0|     3209.0|         379.0|    1199.0|     392.0|      12.2478|          500001.0|         INLAND|
|  -121.89|   37.82|               4.0|    11444.0|        1355.0|    3898.0|    1257.0|      13.2949|          500001.0|         INLAND|
|  -121.91|   37.81|               7.0|     3477.0|         416.0|    1216.0|     395.0|      13.1499|          500001.0|         INLAND|
|  -118.02|   34.17|              32.0|     3868.0|         548.0|    1558.0|     528.0|       9.4667|          500001.0|         INLAND|
|  -118.05|   34.15|              

## Groupby

In [ ]:
from pyspark.sql import functions as F
df.groupBy(["ocean_proximity"]).agg(F.sum("median_house_value").alias("sum_mhvalue"),\
    F.max("median_house_value")).alias("max_mhvalue").show()

+---------------+-------------+-----------------------+
|ocean_proximity|  sum_mhvalue|max(median_house_value)|
+---------------+-------------+-----------------------+
|         ISLAND|    1902200.0|               450000.0|
|     NEAR OCEAN| 6.62995512E8|               500001.0|
|       NEAR BAY| 5.93596194E8|               500001.0|
|      <1H OCEAN|2.193410032E9|               500001.0|
|         INLAND| 8.17600123E8|               500001.0|
+---------------+-------------+-----------------------+



In [ ]:
_from pyspark.sql import functions as F
df.groupBy(["ocean_proximity"]).agg(F.sum("median_house_value") ,F.max("median_house_value")).show()

In [ ]:
#Group by median_income and see the distribution
result = df.groupBy("median_income").count().sort("median_income", ascending=False)
result.show(10)

+-------------+-----+
|median_income|count|
+-------------+-----+
|      15.0001|   48|
|         15.0|    2|
|      14.9009|    1|
|      14.5833|    1|
|      14.4219|    1|
|      14.4113|    1|
|      14.2959|    1|
|      14.2867|    1|
|       13.947|    1|
|      13.8556|    1|
+-------------+-----+
only showing top 10 rows



In [ ]:
df.describe().show()

+-------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+
|summary|          longitude|          latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|        households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+
|  count|              20433|             20433|             20433|             20433|             20433|             20433|             20433|             20433|             20433|          20433|
|   mean|-119.57068859198068| 35.63322125972706|28.633093525179856|2636.5042333480155| 537.8705525375618|1424.9469485635982|499.43346547251997|3.8711616013312273|206864.41315519012|           null|
| stddev| 

In [ ]:
(df.describe().select(
                    "summary",
                    F.round("longitude", 4).alias("longitude"),
                    F.round("latitude", 4).alias("latitude"),
                    F.round("housing_median_age", 4).alias("housing_median_age"),
                    F.round("total_rooms", 4).alias("total_rooms"),
                    F.round("total_bedrooms", 4).alias("total_bedrooms"),
                    F.round("population", 4).alias("population"),
                    F.round("households", 4).alias("households"),
                    F.round("median_income", 4).alias("median_income"),
                    F.round("median_house_value", 4).alias("median_house_value"),
                    F.round("ocean_proximity", 4).alias("ocean_proximity"))
                    .show())

+-------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|summary|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+-------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  count|  20433.0| 20433.0|           20433.0|    20433.0|       20433.0|   20433.0|   20433.0|      20433.0|           20433.0|        20433.0|
|   mean|-119.5707| 35.6332|           28.6331|  2636.5042|      537.8706| 1424.9469|  499.4335|       3.8712|       206864.4132|           null|
| stddev|   2.0036|  2.1363|           12.5918|  2185.2696|      421.3851| 1133.2085|  382.2992|       1.8993|       115435.6671|           null|
|    min|  -124.35|   32.54|               1.0|        2.0|           1.0|       3.0|       1.0|       0.4999|           149

Look at the minimum and maximum values of all the (numerical) attributes. We see that multiple attributes have a wide range of values: we will need to normalize your dataset.

# Data Pre-processing

In [ ]:
#Express the house values in units of 100,000
df = df.withColumn("median_house_value", col("median_house_value")/100000)
df.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|             4.526|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|             3.585|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|             3.521|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|             3.413|       NEAR BAY|
|  -122.25|   37.85|              

# Feature Engineering

Rooms per household which refers to the number of rooms in households per block group;
Population per household, which basically gives us an indication of how many people live in households per block group; And
Bedrooms per room which will give us an idea about how many rooms are bedrooms per block group;

In [ ]:
df = (df.withColumn("rooms_per_household", F.round(col("total_rooms")/col("households"), 2))
                       .withColumn("population_per_household", F.round(col("population")/col("households"), 2))
                       .withColumn("bedrooms_per_room", F.round(col("total_bedrooms")/col("total_rooms"), 2)))
df.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+------------------------+-----------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|rooms_per_household|population_per_household|bedrooms_per_room|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+------------------------+-----------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|             4.526|       NEAR BAY|               6.98|                    2.56|             0.15|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|             3.585|       NEAR BAY|               6.24|                    2.11|             0

# Linear Regression

## Define X and y

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
X = ["total_bedrooms", "population", "households", "median_income", 
     "rooms_per_household", "population_per_household", "bedrooms_per_room"]

assembler = VectorAssembler(inputCols=X, outputCol="X") 

In [ ]:
#Transform all features into Dense Vector
df2 = assembler.transform(df)
df2.show(10, truncate=False)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+------------------------+-----------------+--------------------------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|rooms_per_household|population_per_household|bedrooms_per_room|X                                           |
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+------------------------+-----------------+--------------------------------------------+
|-122.23  |37.88   |41.0              |880.0      |129.0         |322.0     |126.0     |8.3252       |4.526             |NEAR BAY       |6.98               |2.56                    |0.15             |[129.0,322.0,126.0,8.3252,6.98,2.56,0.15]   |
|-122.22  |37.86

## Standardization

In [ ]:
standardScaler = StandardScaler(inputCol="X", outputCol="X_scaled")
scaled_df = standardScaler.fit(df2).transform(df2)
scaled_df.select("X", "X_scaled").show(10, truncate=False)

+--------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+
|X                                           |X_scaled                                                                                                                               |
+--------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+
|[129.0,322.0,126.0,8.3252,6.98,2.56,0.15]   |[0.306133295081709,0.2841489478008318,0.329584763633844,4.383319305578301,2.8111579992311904,0.245252327599291,2.584777863821999]      |
|[1106.0,2401.0,1138.0,8.3014,6.24,2.11,0.16]|[2.6246777082199237,2.1187628064279416,2.976725881073924,4.370788315395149,2.5131269219488006,0.20214156688847812,2.7570963880767994]  |
|[190.0,496.0,177.0,7.2574,8.29,2.8,0.13]    |[0.4508940005079435,0.43769527363109495

## Train test split

In [ ]:
rnd_seed=1024
np.random.seed=rnd_seed
np.random.set_state=rnd_seed

train, test = scaled_df.randomSplit([.8,.2], seed=rnd_seed)

In [ ]:
train.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity',
 'rooms_per_household',
 'population_per_household',
 'bedrooms_per_room',
 'X',
 'X_scaled']

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

lr = (LinearRegression(featuresCol='X_scaled', labelCol="median_house_value", predictionCol='pred_median_house_value', 
                               maxIter=10, regParam=0.3, elasticNetParam=0.8, standardization=False))

model = lr.fit(train)
model.coefficients

DenseVector([0.0, 0.0, 0.0, 0.5297, 0.0, 0.0, 0.0])

In [ ]:
model.intercept

0.9902407246097965

In [ ]:
coeff_df = pd.DataFrame({"Feature": ["Intercept"] + X, "Co-efficients": np.insert(model.coefficients.toArray(), 0, model.intercept)})
coeff_df = coeff_df[["Feature", "Co-efficients"]]
coeff_df

,Feature,Co-efficients
0,Intercept,0.990241
1,total_bedrooms,0.000000
2,population,0.000000
3,households,0.000000
4,median_income,0.529749
5,rooms_per_household,0.000000
6,population_per_household,0.000000
7,bedrooms_per_room,0.000000


In [ ]:
predictions = model.transform(test)
predandlabels = predictions.select("pred_median_house_value", "median_house_value")
predandlabels.show(10)

+-----------------------+------------------+
|pred_median_house_value|median_house_value|
+-----------------------+------------------+
|     1.8310980505664771|             0.946|
|     1.5424168111843946|             0.858|
|     1.6925310556630775|              0.79|
|      1.682099482375261|             0.732|
|     1.4654630392563592|             0.783|
|     1.4512381665911551|             0.581|
|      1.709712470490069|             0.684|
|     1.9765822384521723|             0.901|
|     1.3993592192239404|             0.646|
|      1.797934572921521|             0.864|
+-----------------------+------------------+
only showing top 10 rows



In [ ]:
# The smaller an RMSE value, the closer predicted and observed values are.
print("RMSE: {0}".format(model.summary.rootMeanSquaredError))

RMSE: 0.878548267063143


In [ ]:
print("MAE: {0}".format(model.summary.meanAbsoluteError))

MAE: 0.6755136983514918


In [ ]:
print("R2: {0}".format(model.summary.r2))

R2: 0.4214673669409482


# SPARK SQL!!!!!

In [ ]:
spark.sql("select * from df").show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+------------------------+-----------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|rooms_per_household|population_per_household|bedrooms_per_room|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+------------------------+-----------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|             4.526|       NEAR BAY|               6.98|                    2.56|             0.15|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|             3.585|       NEAR BAY|               6.24|                    2.11|             0

In [ ]:
spark.sql("select rooms_per_household, population_per_household, bedrooms_per_room from df order by median_house_value asc").show()

+-------------------+------------------------+-----------------+
|rooms_per_household|population_per_household|bedrooms_per_room|
+-------------------+------------------------+-----------------+
|               3.57|                    2.79|             0.33|
|              12.25|                    2.25|             0.29|
|               6.71|                    2.24|             0.29|
|               3.77|                    2.99|             0.39|
|               3.57|                    1.88|             0.34|
|               2.25|                    3.28|             0.53|
|               6.93|                    4.75|             0.17|
|               2.02|                    3.15|             0.74|
|                4.7|                    3.43|             0.25|
|               1.63|                    2.37|             0.75|
|                4.9|                    2.75|             0.21|
|                5.7|                    2.95|              0.2|
|               6.47|    

In [ ]:
spark.sql("select total_rooms, median_house_value from df where median_house_value > 5 order by total_rooms").show()

+-----------+------------------+
|total_rooms|median_house_value|
+-----------+------------------+
|        8.0|           5.00001|
|       30.0|           5.00001|
|       42.0|           5.00001|
|       45.0|           5.00001|
|       56.0|           5.00001|
|       58.0|           5.00001|
|       70.0|           5.00001|
|       91.0|           5.00001|
|      102.0|           5.00001|
|      187.0|           5.00001|
|      189.0|           5.00001|
|      204.0|           5.00001|
|      204.0|           5.00001|
|      211.0|           5.00001|
|      238.0|           5.00001|
|      240.0|           5.00001|
|      243.0|           5.00001|
|      249.0|           5.00001|
|      249.0|           5.00001|
|      250.0|           5.00001|
+-----------+------------------+
only showing top 20 rows

